In [3]:
import os

In [4]:

def check_non_empty_json_files(directory):
    # List to hold paths of non-empty JSON files
    non_empty_files = []

    # Walk through the directory structure
    for root, dirs, files in os.walk(directory):
        # print(f'Checking {root}')
        for file in files:
            if file == 'result.json':  # Check if the file is the one we're interested in
                file_path = os.path.join(root, file)  # Full path to the file
                if os.path.getsize(file_path) > 0:  # Check if the file is not empty
                    non_empty_files.append(file_path)

    return non_empty_files


In [5]:

# Define the base directory to start from
base_directory = '/home/users/MTrappett/mtrl/BranchGatingProject/branchNetwork/data/hyper_search/branch_search/'

# Call the function
non_empty_result_json_files = check_non_empty_json_files(base_directory)

# Print out the non-empty files
if non_empty_result_json_files:
    print("Non-empty result.json files found:")
    for file in non_empty_result_json_files:
        print(file)
else:
    print("No non-empty result.json files found.")

No non-empty result.json files found.


[]
